In [ ]:
# 📦 Steg 1: Installera Whisper + Piper
!pip install git+https://github.com/openai/whisper.git
!pip install piper-tts
!apt-get install -y espeak ffmpeg

In [ ]:
# 📥 Steg 2: Ladda ner svensk röstmodell till Piper
!mkdir -p ~/.local/share/piper/voices
!wget -O ~/.local/share/piper/voices/sv-SE-jenny-low.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/sv/sv-SE-jenny-low.onnx

In [ ]:
# 🧠 Steg 3: Ladda MythoMax
!pip install transformers accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'TheBloke/MythoMax-L2-13B-GGUF'
tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B')
model = AutoModelForCausalLM.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B')

In [ ]:
# 🎤 Steg 4: Lyssna på mikrofon och transkribera med Whisper
import whisper
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

print("🎤 Säg något till Pia...")
duration = 5
fs = 16000
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
sd.wait()
wav.write("temp.wav", fs, recording)
model_whisper = whisper.load_model("base")
result = model_whisper.transcribe("temp.wav")
input_text = result["text"]

In [ ]:
# 💬 Steg 5: Generera svar med modellen
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
response = pipe(input_text, max_new_tokens=100)[0]['generated_text']
print("Pia:", response)

In [ ]:
# 🔊 Steg 6: Säg svaret med röst
import os
os.system(f'piper --model ~/.local/share/piper/voices/sv-SE-jenny-low.onnx --text "{response}" --output_file out.wav')
os.system("ffplay -nodisp -autoexit out.wav")